In [ ]:
from keras.layers import GlobalAvgPool2D
from keras.layers import MaxPool2D
from keras.layers import AvgPool2D
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import DepthwiseConv2D
from keras.layers import BatchNormalization
from keras.layers import ReLU
from keras.layers import Dense
from keras.layers import Concatenate
from keras.layers.merge import concatenate
from keras.backend import int_shape

In [ ]:
input_layer = Input(shape=(224,224, 3))
model = Create_DenseNet121_Model(input_layer,1000)
model.summary()

In [ ]:
def Create_DenseNet121_Model(input_layer,number_classes):
  
  filter = 32
  layer1 = Conv2D(64,(7,7), strides = (2,2) , padding='same')(input_layer)
  dense_block1 = MaxPool2D((3,3),strides=(2,2), padding='same' )(layer1)

  # dense_block 1
  for i in range(6):
    dense_block1 =  dense_layer(dense_block1,filter)
  
  #transition_layer  
  dense_block2 = transition_layer(dense_block1)

  # dense_block 2
  for i in range(12):
    dense_block2 = dense_layer(dense_block2,filter)

  #transition_layer  
  dense_block3 = transition_layer(dense_block2)

  # dense_block 3
  for i in range(24):
    dense_block3 =  dense_layer(dense_block3,filter)

  #transition_layer  
  dense_block4 = transition_layer(dense_block3)
  
  # dense_block 4
  for i in range(16):
    dense_block4 =  dense_layer(dense_block4,filter)
  
  pool = GlobalAvgPool2D()(dense_block4)
  
  output = Dense(number_classes, activation='softmax')(pool)
  
  model = Model(input_layer, output)
  return model


In [ ]:
def transition_layer(input_layer):
  
  norm = BatchNormalization()(input_layer)
  relu = ReLU()(norm)
  # print(int_shape(input_layer)[-1])
  conv1 = Conv2D(int_shape(input_layer)[-1]//2,(1,1),padding='same')(relu)

  output = AvgPool2D( 2, strides=2, padding='same')(conv1)

  return output

def dense_layer(input_layer,filter):

  norm1 = BatchNormalization()(input_layer)
  relu1 = ReLU()(norm1)
  conv1 = Conv2D(4 * filter,(1,1),padding='same')(relu1)

  norm3 = BatchNormalization()(conv1)
  relu3 = ReLU()(norm3)
  conv3 = Conv2D(filter,(3,3),padding='same')(relu3)

  output = Concatenate()([ input_layer, conv3])

  return output
